### Classifcation Module

In [ ]:
from tensorflow import keras
import tensorflow as tf

import numpy as np
from PIL import Image

In [3]:
# load model
model = keras.models.load_model('Models/model.h5')
print(model)

In [4]:
classes = ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [ ]:
img_height = 180
img_width = 180

In [ ]:
# Image verification
def verify(image_path):
    # 1. Read the images using PIL
    image = Image.open(image_path)

    # 2. Resize the image to 256x256 pixels each using PIL
    image = image.resize((256, 256))

    # 3. Store pixel values of image in a list using PIL
    image_pixels = list(image.getdata())

    # 4. Check if green pixel values exist in stored list
    green_pixels = [pixel for pixel in image_pixels if pixel[1] > 200]

    # 5. If green pixel values exist, return True
    if len(green_pixels) > 0:
        return True
    # 6. If green pixel values do not exist, return False & print error message
    else:
        return False

In [ ]:
def predict(image_path):
    img = tf.keras.utils.load_img(
        image_path, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    label = classes[np.argmax(score)]
    confidence = 100 * np.max(score)

    return label, confidence

### Segmentation Module

In [ ]:
def verify_seg(label):
    if label != 'healthy':
        return True
    else:
        return False

In [ ]:
def segment(image_path):
    if verify_seg = True:
        # execute segmentation
        return 0

### Combined Module